# Contextualized Prompts 

<a target="_blank" href="https://colab.research.google.com/github/tecton-ai/gen-ai/blob/main/context-aware-prompt.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="150"/>
</a>

This tutorial guides you through creating an LLM generated restaurant recommendation function.
This is an example of how Tecton managed and contextualized prompts enable personalization.

It uses Tecton's real-time enriched prompts to provide current context to the LLM in order to improve the quality of its response. 
This tutorial demonstrates both LangChain and LlamaIndex integration with Tecton prompts.

## Install Packages

In [ ]:
!pip install 'tecton-gen-ai[tecton,langchain,llama-index]' langchain-openai llama-index-llms-openai

## Log in to Tecton
Make sure to hit enter after pasting in your authentication token.

In [ ]:
import tecton

tecton.login("explore.tecton.ai")
tecton.set_validation_mode("auto")

# Tecton Prompt

In the following cell you'll create a Tecton Agent with a system prompt that provides instructions to the LLM. The instructions are parameterized with a specific user's data. 

The agent creation function takes a Tecton feature view as input which is used at run-time to acquire the latest values of the parameters for the user.

In [7]:
from tecton_gen_ai.agent import AgentClient, AgentService
from tecton_gen_ai.fco import prompt
from tecton_gen_ai.core_utils import make_request_source


def restaurant_recommender_agent( user_info):    
    
    location_request = make_request_source(location = str)

    @prompt(sources=[ location_request, user_info])
    def sys_prompt(location_request, user_info ):
        name = user_info["name"]
        food_preference = user_info["food_preference"]
        location = location_request["location"]
        return f"""
        You are a consierge service that recommends restaurants.
        You are serving {name}. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in {location}. 
        If the user does not provide a cuisine or food preference, choose a {food_preference} restaurant.
        """
        
    return AgentService(
        name="restaurant_recommender",
        prompts=[ sys_prompt],
        tools=[user_info],
    )

The example above uses a single feature view as input. Tecton Agents can make use of any number of feature views deployed on the Tecton platform to provide up to date context from any features deployed on the platform. 

Notice that the `sys_prompt` function additionally takes the `location` parameter in the prompt. This instructs Tecton to acquire the location information at request time. Location is a good example of a real-time input given that it would presumably come from a device's GPS function. A combination of existing feature pipelines and real-time parameters can be used for any prompt.

## Sample Data

In order to keep this notebook self-contained, you will create a mock feature view with some hard-coded data.
In a real application, you would use Feature Views that continuously update feature values and therefore provide up-to-date context to the LLM application.

In [5]:
import pandas as pd
from tecton import RequestSource
from tecton.types import Field, String


from tecton_gen_ai.testing import mock_batch_feature_view


mock_data = pd.DataFrame(
        [
            {
                "user_id": "user1",
                "name": "Jim",
                "age": 30,
                "food_preference": "American",
            },
            {
                "user_id": "user2",
                "name": "John",
                "age": 40,
                "food_preference": "Italian",
            },
            {
                "user_id": "user3",
                "name": "Jane",
                "age": 50,
                "food_preference": "Chinese",
            },
        ]
    )

user_preference_fv = mock_batch_feature_view(
        "user_info", mock_data, entity_keys=["user_id"], description="User's profile with name, age and food preference."
    )

The feature view identifies the key `user_id` that is needed to access a user's data, this attribute must be provided when using the feature view in a prompt. 

In the following cell, you will test the prompt through an AgentClient's invoke_prompt method using a `user_id` and a `location` value. The `user_id` is used to retrieve a specific user's values. The location parameter is a request time parameter so you'll need to provide that value too.

In [8]:
# create the Tecton Agent
recommender_agent = restaurant_recommender_agent(user_preference_fv )

# create a client to invoke with the agent
client = AgentClient.from_local( recommender_agent )

#test the agent using "sys_prompt" prompt
print(client.invoke_prompt("sys_prompt", kwargs=dict(user_id="user3", location="Chicago")))



        You are a consierge service that recommends restaurants.
        You are serving Jane. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in Chicago. 
        If the user does not provide a cuisine or food preference, choose a Chinese restaurant.
        


## Incorporate Contextualized Prompt into a LangChain agent

The Tecton AgentClient can be used to create a LangChain agent which will use the enriched prompt to generate a response.
In the cell below you will instantiate an LLM model using OpenAI.

Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.

In [9]:
import openai as oa
import os
from langchain_openai import ChatOpenAI


# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# instantiate LLM model
gpt_llm = ChatOpenAI(model="gpt-4o-mini")

#create a lang chain agent that uses the system_prompt 
lc_agent = client.make_agent(llm=gpt_llm, system_prompt = "sys_prompt")

## Test it out

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

In [10]:
with client.set_context({"user_id":"user1", "location":"Charlotte, NC"}):
    print(lc_agent.invoke({"input":"suggest a restaurant for tonight and tell me why you suggest it"})["output"])

Hi Jim! I recommend trying out **The Capital Grille** in Charlotte, NC for dinner tonight. 

This restaurant is known for its upscale American cuisine and an extensive wine list. The atmosphere is elegant, making it perfect for a special night out. 

I suggest trying their **Dry Aged Porterhouse** or the **Seared Tenderloin with Butter Poached Lobster**. Both dishes are highly praised and are sure to satisfy your taste. For a side, their **Truffle Fries** are a must-try! 

Enjoy your dining experience!


In [11]:
with client.set_context({"user_id":"user1", "location":"New York, NY"}):
    print(lc_agent.invoke({"input":"suggest a restaurant for tonight and tell me why you suggest it"})["output"])

Hi Jim! I recommend you try "The Smith," which is a popular American restaurant located in New York, NY. 

The Smith offers a vibrant atmosphere and is known for its delicious comfort food. I suggest trying their famous mac and cheese, the crispy chicken, or the classic cheeseburger. They also have fantastic cocktails if you're in the mood for a drink. 

It's a great spot for a casual yet enjoyable dining experience. Enjoy your dinner!


In [12]:
with client.set_context({"user_id":"user2", "location":"New York, NY"}):
    print(lc_agent.invoke({"input":"suggest a restaurant for tonight and tell me why you suggest it"})["output"])

Hi John! I recommend you try **Carbone**, an iconic Italian restaurant in New York, NY. 

Carbone is known for its classic Italian-American dishes and a vibrant, upscale atmosphere. Their menu features delicious items like the Spicy Rigatoni Vodka, Veal Parmesan, and the famous Tiramisu for dessert. The combination of great food, attentive service, and a lively ambiance makes it a fantastic choice for a night out. Enjoy your dinner!


In [13]:
with client.set_context({"user_id":"user3", "location":"Charlotte, NC"}):
    print(lc_agent.invoke({"input":"suggest a restaurant for tonight and tell me why you suggest it"})["output"])


Hi Jane! I recommend you try "Lang Van," a fantastic Chinese restaurant in Charlotte, NC. 

Lang Van is known for its authentic flavors and cozy atmosphere. One of their standout dishes is the "Kung Pao Chicken," which is a delightful mix of tender chicken, peanuts, and vegetables in a savory sauce. You might also enjoy their "Hot and Sour Soup," which is perfect for a warm starter. 

The combination of great food and inviting service makes it a wonderful choice for your dinner tonight! Enjoy!


## Incorporate Contextualized Prompt into a LlamaIndex agent

The Tecton AgentClient can also be used to create a LlamaIndex agent which will use the enriched prompt to generate a response.
In the cell below you will instantiate an LLM model but this time using LlamaIndex's integration with OpenAI.

In [14]:
import openai as oa
import os
from llama_index.llms.openai import OpenAI

# instantiate LLM model
gpt_llm = OpenAI(model="gpt-4o-mini")

## Test it out

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

Notice that the LlamaIndex agent `li_agent`uses the `chat` method vs LangChain's `invoke` method. 

In [15]:
#create a llama-index agent that uses the system_prompt 
li_agent = client.make_agent(llm=gpt_llm, system_prompt = "sys_prompt")

# context: user1 in Charlotte
with client.set_context({"user_id":"user1", "location":"Charlotte, NC"}):
    print(li_agent.chat("suggest a restaurant for tonight and tell me why you suggest it"))

Hi Jim! I recommend you try **The Capital Grille** in Charlotte, NC. 

This upscale steakhouse is known for its dry-aged steaks and extensive wine list. The ambiance is perfect for a nice evening out, whether it's a special occasion or just a treat for yourself. 

I suggest trying their **Bone-In Ribeye** or the **Filet Mignon**, paired with their **Truffle Fries**. For dessert, don't miss the **Chocolate Cake**—it's a crowd favorite! 

Enjoy your dinner!


In [16]:
# since llama-index chat is stateful, you should create another instance if there is a change in context
li_agent = client.make_agent(llm=gpt_llm, system_prompt = "sys_prompt")

# context: user1 in New York
with client.set_context({"user_id":"user1", "location":"New York, NY"}):
    print(li_agent.chat("suggest a restaurant for tonight and tell me why you suggest it"))

Hi Jim! I recommend you try **The Smith** in New York, NY. It's a fantastic American brasserie known for its vibrant atmosphere and delicious comfort food. 

You might want to try their famous **Mac & Cheese**, which is a crowd favorite, or the **Smith Burger** for a classic taste. If you're in the mood for something lighter, the **Roasted Chicken** is also a great choice. 

The ambiance is lively, making it perfect for a fun night out. Enjoy your dinner!


In [17]:
# since llama-index chat is stateful, you should create another instance if there is a change in context
li_agent = client.make_agent(llm=gpt_llm, system_prompt = "sys_prompt")

# context: user2 in Charlotte
with client.set_context({"user_id":"user2", "location":"Charlotte, NC"}):
    print(li_agent.chat("suggest a restaurant for tonight and tell me why you suggest it"))

I recommend trying **Caffe Siena** in Charlotte, NC. This charming Italian restaurant offers a cozy atmosphere and a menu filled with authentic Italian dishes. 

You might enjoy their **Fettuccine Alfredo**, which is creamy and rich, or the **Margherita Pizza**, made with fresh mozzarella and basil. For a delightful dessert, don't miss their **Tiramisu**, which is a perfect way to end your meal. 

Caffe Siena is known for its warm service and inviting ambiance, making it a great choice for a lovely dinner tonight. Enjoy your meal, John!


In [18]:
# since llama-index chat is stateful, you should create another instance if there is a change in context
li_agent = client.make_agent(llm=gpt_llm, system_prompt = "sys_prompt")

# context: user3 in Charlotte
with client.set_context({"user_id":"user3", "location":"Charlotte, NC"}):
    print(li_agent.chat("suggest a restaurant for tonight and tell me why you suggest it"))


Hi Jane! I recommend you try **Lang Van** in Charlotte, NC. It's a fantastic Chinese restaurant known for its authentic flavors and cozy atmosphere.

Here are a few menu items you might enjoy:
- **Kung Pao Chicken**: A classic dish with a perfect balance of spicy and savory flavors.
- **Beef with Broccoli**: Tender beef stir-fried with fresh broccoli in a delicious sauce.
- **Shrimp Fried Rice**: A flavorful and satisfying dish that pairs well with any entrée.

Lang Van is well-loved for its quality ingredients and friendly service, making it a great choice for a delightful dining experience tonight! Enjoy your meal!


# Conclusion

Tecton prompts are used to incorporate real-time, streaming and batch features into your generative AI applications, providing a great solution for personalization. In general, it can be used to provide up to date context for any LLM driven function and ut provides seamless integration with LangChain and LlamaIndex. 
